In [1]:
import os
gpu = os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
import warnings
warnings.filterwarnings("ignore")
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer,pipeline, logging, BitsAndBytesConfig, AutoModelForCausalLM
from datasets import load_dataset
from random import randrange

2024-05-07 12:40:37.420051: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-07 12:40:37.507357: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 12:40:39.018298: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [13]:
# Get the model
# The model that you want to train from the Hugging Face hub
# model_name = "meta-llama/Llama-2-7b-chat-hf"
model_name = "meta-llama/Llama-2-7b-chat-hf"

In [14]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [ ]:
DEFAULT_SYSTEM_PROMPT = """You are a fine-tuned AI model who is a math genious. 
You can solve simple to moderate level mathematics problems. 
Follow a chain of thought approach while answering. Answer in brief. """

In [15]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [16]:
# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0},
    cache_dir="/projects/barman/cache",
#     use_flash_attention_2=use_flash_attention,
)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.69s/it]


In [18]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [21]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
# Run text generation pipeline with our next model
prompt = "Can you explain Caeser Cipher with an example?"
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=512)
result = pipe(f"<s>[INST] {prompt} [/INST]")

print(f"Input:\n{prompt}\n")
print(f"Generated Response llama 2 7b chat\n {result[0]['generated_text'].split("[/INST]")[-1]}\n")

Input:
Can you explain Caeser Cipher with an example?

Generated Response llama 2 7b chat
   Sure, I'd be happy to explain the Caesar Cipher!
 Unterscheidung:

The Caesar Cipher is a simple encryption technique that replaces each letter of a message with a letter a fixed number of positions down the alphabet. For example, if we use a shift of 2, the letter "a" becomes "c", "b" becomes "d", "c" becomes "e", and so on.

Here's an example of how the Caesar Cipher works:

Original message: HELLO
Shift: 2
Encrypted message: CJLG

To encrypt the message "HELLO", we shift each letter 2 positions down the alphabet, so the first letter "H" becomes "C", the second letter "E" becomes "G", and so on.

Decryption is simply the reverse of encryption, where we shift each letter back up the alphabet by the same number of positions. In this case, the decrypted message is "HELLO".

Here's another example:

Original message: THE QUICK BROWN FOX
Shift: 3
Encrypted message: KCJG UBXF

In this example, the 

In [22]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
# Run text generation pipeline with our next model
prompt = "Can you explain Caeser Cipher with an example?"
SYSTEM_PROMPT = "You are a cryptographic expert and you can easily solve medium to higher difficulty cryptographic problems. Please follow my instructions and solve the problem using Chain of thought approach."
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=512)
result = pipe(f"<s> <<SYS>> {SYSTEM_PROMPT} <</SYS>> [INST] {prompt} [/INST]")

print(f"Input:\n{prompt}\n")
print(f"Generated Response llama 2 7b chat\n {result[0]['generated_text'].split("[/INST]")[-1]}\n")


Input:
Can you explain Caeser Cipher with an example?

Generated Response llama 2 7b chat
   Of course! The Caesar Cipher is a simple encryption technique that replaces each letter of a message with a letter a fixed number of positions down the alphabet. Here's how it works:

1. Choose a number of positions to shift the letters by (e.g. 3). This is called the "cipher shift."
2. Write out the message you want to encrypt, using each letter as is.
3. To encrypt each letter, move it down the alphabet by the number of positions you chose in step 1. For example, if the message is "HELLO", and you chose a cipher shift of 3, the encrypted message would be "KLLO".

Here's an example of how to encrypt the message "HELLO" using a cipher shift of 3:

H -> K (position 3)
E -> F (position 6)
L -> L (position 3)
L -> K (position 6)
O -> O (position 0)

So the encrypted message is "KLLO".

To decrypt the message, you simply reverse the process by moving each letter up the alphabet by the same number o

In [23]:
# Does Few Shot learning helps in any way ?
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
# Run text generation pipeline with our next model
prompt = """Can you explain Caeser Cipher with an example? 
For example: If the given world is HELLO and shift value is 2 then each letter should sshift by 2 towards the right from the English Alphabet, 
i.e. H becomes J, E becomes G, L becomes N, L becomes N, O becomes Q. Similarly, the word ART with a shift of -4 becomes WNP. """
SYSTEM_PROMPT = "You are a cryptographic expert and you can easily solve medium to higher difficulty cryptographic problems. Please follow my instructions and solve the problem using Chain of thought approach."
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=512)
result = pipe(f"<s> <<SYS>> {SYSTEM_PROMPT} <</SYS>> [INST] {prompt} [/INST]")

print(f"Input:\n{prompt}\n")
print(f"Generated Response llama 2 7b chat\n {result[0]['generated_text'].split("[/INST]")[-1]}\n")


Input:
Can you explain Caeser Cipher with an example? 
For example: If the given world is HELLO and shift value is 2 then each letter should sshift by 2 towards the right from the English Alphabet, 
i.e. H becomes J, E becomes G, L becomes N, L becomes N, O becomes Q. Similarly, the word ART with a shift of -4 becomes WNP 

Generated Response llama 2 7b chat
   Of course! I'd be happy to explain the Caesar Cipher using an example.

The Caesar Cipher is a simple encryption technique that replaces each letter of a message with a letter a fixed number of positions down the alphabet. The encryption is done by shifting each letter of the message by a certain number of positions, either towards the right (positive shift) or towards the left (negative shift).

Let's use the example you provided:

Original message: HELLO
Shift value: 2

To encrypt the message "HELLO", we shift each letter by 2 positions towards the right in the alphabet:

Encrypted message: JGNO

Here's how the encryption work

In [25]:
# Is there a problem with the llama tokenizer?
# Test tokenizer on the prompt

sentence = "If the given world is HELLO and shift value is 2 then each letter should sshift by 2 towards the right from the English Alphabet, i.e. H becomes J, E becomes G, L becomes N, L becomes N, O becomes Q. Similarly, the word ART with a shift of -4 becomes WNP."
encoded_output = tokenizer.encode(sentence)
print(f"Original sentence: {sentence}")
print(f"Encoded sentence: {encoded_output}")

# Verify what each token ID correlates to
for token in encoded_output:
    print(f"Token ID {token} --> {tokenizer.decode(token)}")


Original sentence: If the given world is HELLO and shift value is 2 then each letter should sshift by 2 towards the right from the English Alphabet, i.e. H becomes J, E becomes G, L becomes N, L becomes N, O becomes Q. Similarly, the word ART with a shift of -4 becomes WNP.
Encoded sentence: [1, 960, 278, 2183, 3186, 338, 17714, 2208, 29949, 322, 9500, 995, 338, 29871, 29906, 769, 1269, 5497, 881, 269, 10889, 491, 29871, 29906, 7113, 278, 1492, 515, 278, 4223, 838, 17416, 29892, 474, 29889, 29872, 29889, 379, 7415, 435, 29892, 382, 7415, 402, 29892, 365, 7415, 405, 29892, 365, 7415, 405, 29892, 438, 7415, 660, 29889, 20175, 29892, 278, 1734, 9033, 29911, 411, 263, 9500, 310, 448, 29946, 7415, 399, 25500, 29889]
Token ID 1 --> <s>
Token ID 960 --> If
Token ID 278 --> the
Token ID 2183 --> given
Token ID 3186 --> world
Token ID 338 --> is
Token ID 17714 --> HE
Token ID 2208 --> LL
Token ID 29949 --> O
Token ID 322 --> and
Token ID 9500 --> shift
Token ID 995 --> value
Token ID 338 --> is

In [26]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
# Run text generation pipeline with our next model
prompt = "Define plaintext and ciphertext in cryptography."
SYSTEM_PROMPT = "You are a cryptographic expert and you can easily solve medium to higher difficulty cryptographic problems. Please follow my instructions and solve the problem using Chain of thought approach."
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=512)
result = pipe(f"<s> <<SYS>> {SYSTEM_PROMPT} <</SYS>> [INST] {prompt} [/INST]")

print(f"Input:\n{prompt}\n")
print(f"Generated Response llama 2 7b chat\n {result[0]['generated_text'].split("[/INST]")[-1]}\n")

Input:
Define plaintext and ciphertext in cryptography.

Generated Response llama 2 7b chat
   In cryptography, plaintext refers to the original, unencrypted message or data that is intended to be protected from unauthorized access. This is the message that is easily readable and understandable by its intended recipient.

On the other hand, ciphertext is the encrypted version of the plaintext message. It is the scrambled, encoded, or encrypted message that can only be decrypted and read by someone who has the appropriate decryption key or algorithm. In other words, ciphertext is the encrypted message that is not easily readable or understandable by anyone who does not have the proper authorization to access it.

To illustrate the difference between plaintext and ciphertext, let's consider an example:

Suppose you want to send an email to a colleague, and you want to ensure that only your colleague can read the contents of the email. In this case, the email contents are the plaintext me

In [27]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
# Run text generation pipeline with our next model
prompt = "Define plaintext and ciphertext in cryptography."
# SYSTEM_PROMPT = "You are a cryptographic expert and you can easily solve medium to higher difficulty cryptographic problems. Please follow my instructions and solve the problem using Chain of thought approach."
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=512)
result = pipe(f"<s> [INST] {prompt} [/INST]")

print(f"Input:\n{prompt}\n")
print(f"Generated Response llama 2 7b chat\n {result[0]['generated_text'].split("[/INST]")[-1]}\n")

Input:
Define plaintext and ciphertext in cryptography.

Generated Response llama 2 7b chat
   In cryptography, plaintext and ciphertext are two fundamental concepts used to describe the process of encryption and decryption.

Plaintext refers to the original, unencrypted text or data that is intended to be protected from unauthorized access or interception. In other words, it is the original message or data that is meant to be encrypted. Plaintext is the "clear" or "unencoded" version of the data, and it is the starting point for the encryption process.

Ciphertext, on the other hand, refers to the encrypted version of the plaintext. It is the text or data that has been transformed through an encryption algorithm, making it unreadable to anyone who does not have the appropriate decryption key or method. In other words, ciphertext is the "encoded" or "encrypted" version of the plaintext, and it is the result of the encryption process.

To illustrate the difference between plaintext and 

In [35]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
# Run text generation pipeline with our next model
prompt = "What is the Vigenère cipher? Answer is brief."
# SYSTEM_PROMPT = "You are a cryptographic expert and you can easily solve medium to higher difficulty cryptographic problems. Please follow my instructions and solve the problem using Chain of thought approach."
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=512)
result = pipe(f"<s> [INST] {prompt} [/INST]")

print(f"Input:\n{prompt}\n")
print(f"Generated Response llama 2 7b chat\n {result[0]['generated_text'].split("[/INST]")[-1]}\n")

Input:
What is the Vigenère cipher? Answer is brief.

Generated Response llama 2 7b chat
   The Vigenère cipher is a polyalphabetic substitution cipher that uses a series of Caesar ciphers based on a keyword. Each letter of the plaintext is replaced by a letter a fixed number of positions down the alphabet, and the same substitution is applied to each letter of the plaintext. The number of positions is determined by the keyword.



In [48]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
# Run text generation pipeline with our next model
prompt = "What is the main difference between substitution-permutation networks (SPN) and Feistel networks in block cipher design? Answer very briefly"
# SYSTEM_PROMPT = "You are a cryptographic expert and you can easily solve medium to higher difficulty cryptographic problems. Please follow my instructions and solve the problem using Chain of thought approach."
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=512)
result = pipe(f"<s> [INST] {prompt} [/INST]")

print(f"Input:\n{prompt}\n")
print(f"Generated Response llama 2 7b chat\n {result[0]['generated_text'].split("[/INST]")[-1]}\n")

Input:
What is the main difference between substitution-permutation networks (SPN) and Feistel networks in block cipher design? Answer very briefly

Generated Response llama 2 7b chat
   Substitution-permutation networks (SPN) and Feistel networks are both block cipher structures used for encryption. Unterscheidung between them lies in the way they combine substitution and permutation operations:

* SPN: Substitution operations are applied first, followed by permutation operations.
* Feistel network: Permutation operations are applied first, followed by substitution operations.

In simpler terms, SPNs perform substitution and permutation operations in a particular order, while Feistel networks perform permutation and substitution operations in a mixed order.



In [49]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
# Run text generation pipeline with our next model
prompt = "Is 989797798979897988 divisible by 3? Justify your answer in brief"
# SYSTEM_PROMPT = "You are a cryptographic expert and you can easily solve medium to higher difficulty cryptographic problems. Please follow my instructions and solve the problem using Chain of thought approach."
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=512)
result = pipe(f"<s> [INST] {prompt} [/INST]")

print(f"Input:\n{prompt}\n")
print(f"Generated Response llama 2 7b chat\n {result[0]['generated_text'].split("[/INST]")[-1]}\n")

Input:
Is 989797798979897988 divisible by 3? Justify your answer in brief

Generated Response llama 2 7b chat
   No, 989797798979897988 is not divisible by 3. everybody.

To check if a number is divisible by 3, we can simply divide the number by 3 and see if the result is a whole number. In this case, dividing 989797798979897988 by 3 results in a remainder of 0, which means that 3 does not divide into the number. Therefore, 989797798979897988 is not divisible by 3.



In [50]:
# Is the llama tokenizer at fault?
# Test tokenizer on the prompt

sentence = "Is 989797798979897988 divisible by 3?"
encoded_output = tokenizer.encode(sentence)
print(f"Original sentence: {sentence}")
print(f"Encoded sentence: {encoded_output}")

# Verify what each token ID correlates to
for token in encoded_output:
    print(f"Token ID {token} --> {tokenizer.decode(token)}")


Original sentence: Is 989797798979897988 divisible by 3?
Encoded sentence: [1, 1317, 29871, 29929, 29947, 29929, 29955, 29929, 29955, 29955, 29929, 29947, 29929, 29955, 29929, 29947, 29929, 29955, 29929, 29947, 29947, 8572, 1821, 491, 29871, 29941, 29973]
Token ID 1 --> <s>
Token ID 1317 --> Is
Token ID 29871 --> 
Token ID 29929 --> 9
Token ID 29947 --> 8
Token ID 29929 --> 9
Token ID 29955 --> 7
Token ID 29929 --> 9
Token ID 29955 --> 7
Token ID 29955 --> 7
Token ID 29929 --> 9
Token ID 29947 --> 8
Token ID 29929 --> 9
Token ID 29955 --> 7
Token ID 29929 --> 9
Token ID 29947 --> 8
Token ID 29929 --> 9
Token ID 29955 --> 7
Token ID 29929 --> 9
Token ID 29947 --> 8
Token ID 29947 --> 8
Token ID 8572 --> divis
Token ID 1821 --> ible
Token ID 491 --> by
Token ID 29871 --> 
Token ID 29941 --> 3
Token ID 29973 --> ?
